In [213]:
# fill out the API key as a string here
API_key = "348f5da85d9669c1f1ed427e95c6b9cd"

In [214]:
import hashlib
import requests
import json
import os

#generate the hash for the file in SHA-256 based on the file contents

# change the file here
file = "HuckleBerryFinn.txt"
# file = "samplefile.txt"

hashfunction = hashlib.sha256()
body = False

In [215]:
# compute hash of body of file instead of the filename
if body:
    for line in open(file):
        hashfunction.update(bytes(line,'utf-8'))
# compute hash using filename
elif not body:
    line = file
    hashfunction.update(bytes(line,'utf-8'))

In [216]:
hash_number = hashfunction.hexdigest()
print(hash_number)

61ab906cb7e4cc50fd90a968d5f93ad8af93ac2faa9dab8a4924e5a17f5b7814


In [217]:
# using API key, request GET of hash scan from api
headers = {'apikey': API_key}
hash_scan = requests.request("GET", f"https://api.metadefender.com/v4/hash/{hash_number}", headers = headers)


In [218]:
# test/understanding code to see what hash_scan returns and how to use it
# print(hash_scan)
# scan_info = json.loads(hash_scan.text)
# print(scan_info)
# for k,v in scan_info["scan_results"]['scan_details'].items():
#     print(str(k) + ": " + str(v))

In [219]:
# get file payload here
script_dir = os.path.abspath(file)
script_dir = f"\"@{script_dir}"

In [248]:
# Now if hash scan doesn't exist scan and upload the file 

if hash_scan.status_code == 404:
#     change the content type of the file here
    content_type = "application/octet-stream"
    headers = {"apikey": API_key, "Content-Type" : content_type, "filename": f"{file}"}
#     scan a file, returns a json object with data_id
    payload = script_dir
    file_scan = requests.post('https://api.metadefender.com/v4/file', headers = headers, data = payload)

In [249]:
# test/understanding code to see what file scan returns and how to use it
if hash_scan.status_code == 404:
    print(file_scan.text)

{"data_id":"bzIyMDkwNy0zY3hsRWtVU0FsYUZHbEZYdS1J","status":"inqueue","in_queue":1,"queue_priority":"normal","sha1":"D781281ACEC5C725A77573FDCCFD4729BAE6BF32","sha256":"35F905F5C37919F702722CF3144653E98B152613CD67F94B9DB28DA9786030A0"}


In [250]:
if hash_scan.status_code == 404:
    headers = {'apikey': API_key}
    data_id = json.loads(file_scan.text)["data_id"]
    results = requests.get(f'https://api.metadefender.com/v4/file/{data_id}', headers = headers)
#     repeatedly pull till progress is 100 or safety counter
    stop = 0
    while json.loads(results.text)["scan_results"]["progress_percentage"] < 100 and stop < 10:
        results = requests.get(f'https://api.metadefender.com/v4/file/{data_id}', headers = headers)
        stop += 1

In [251]:
# check the results text values
if hash_scan.status_code == 404:
    print(results.text)

{"appinfo":false,"data_id":"bzIyMDkwNy0zY3hsRWtVU0FsYUZHbEZYdS1J","scan_results":{"scan_details":{},"scan_all_result_i":254,"start_time":"2022-09-07T05:40:16.459Z","total_time":0,"total_avs":0,"total_detected_avs":0,"progress_percentage":0,"scan_all_result_a":"In queue"},"file_info":{"file_size":0,"upload_timestamp":"","md5":"","sha1":"","sha256":"","file_type_category":"","file_type_description":"","file_type_extension":"","display_name":""},"share_file":1,"private_processing":0,"rest_version":"4","additional_info":[],"votes":{"up":0,"down":0},"stored":false}
50


In [224]:
# previous step skipped if scan exists, display the results at the very end
if hash_scan.status_code == 200:
#     if hash_scan worked then store values into results
    results = hash_scan
#     load into a results dictionary
results_dict = json.loads(results.text)
threats = 0
engine_text = ""
# go through scan details and store printable values into string
for k,v in results_dict["scan_results"]['scan_details'].items():
    engine_text += "engine: " + str(k) + "\n"
    if v["threat_found"] == "":
        engine_text += "threat_found: Clean\n"
    else:
#         if there is a threat we want to store number of threats for overall status
        engine_text += "threat_found: " + str(v["threat_found"]) + "\n"
        threats += 1
    engine_text += "scan result: " + str(v["scan_result_i"]) + "\n"
    engine_text += "def_time: " + str(v["def_time"]) + "\n"
    
    
print("filename: " + str(file))
# overall status is clean if zero threats
if threats == 0:
    print("overall status: " + "Clean")
else:
    print("overall status: " + " Not Clean")
print(engine_text)

filename: HuckleBerryFinn.txt
overall status: Clean



In [233]:
def run_program(API_key:str, file_name:str):
    file = file_name
    hashfunction = hashlib.sha256()
    body = False
    # compute hash of body of file instead of the filename
    if body:
        for line in open(file):
            hashfunction.update(bytes(line,'utf-8'))
    # compute hash using filename
    elif not body:
        line = file
        hashfunction.update(bytes(line,'utf-8'))
    hash_number = hashfunction.hexdigest()
    headers = {'apikey': API_key}
    hash_scan = requests.request("GET", f"https://api.metadefender.com/v4/hash/{hash_number}", headers = headers)
    if hash_scan !=  200 or hash_scan != 404:
        print("hash_scan has returned " + str(hash_scan) + " which is not 200 or 404. Ending function")
        return
    script_dir = os.path.abspath(file)
    script_dir = f"\"@{script_dir}"
    if hash_scan.status_code == 404:
        content_type = "application/octet-stream"
        headers = {"apikey": API_key, "Content-Type" :content_type}
        payload = script_dir
        file_scan = requests.post('https://api.metadefender.com/v4/file', headers = headers, data = payload)
        if file_scan !=  200 or file_scan != 404:
            print("file_scan has returned " + str(file_scan) + " which is not 200 or 404. Ending function")
            return
        headers = {'apikey': API_key}
        data_id = json.loads(file_scan.text)["data_id"]
        results = requests.get(f'https://api.metadefender.com/v4/file/{data_id}', headers = headers)
        if results !=  200 or results != 404:
            print("results has returned " + str(results) + " which is not 200 or 404. Ending function")
            return
        stop = 0
        while json.loads(results.text)["scan_results"]["progress_percentage"] < 100 and stop < 10:
            if results !=  200 or results != 404:
                print("results has returned " + str(results) + " which is not 200 or 404. Ending function")
                return
            results = requests.get(f'https://api.metadefender.com/v4/file/{data_id}', headers = headers)
            stop += 1
    if hash_scan.status_code == 200:
        results = hash_scan
    results_dict = json.loads(results.text)
    threats = 0
    engine_text = ""
    for k,v in results_dict["scan_results"]['scan_details'].items():
        engine_text += "engine: " + str(k) + "\n"
        if v["threat_found"] == "":
            engine_text += "threat_found: Clean\n"
        else:
            engine_text += "threat_found: " + str(v["threat_found"]) + "\n"
            threats += 1
        engine_text += "scan result: " + str(v["scan_result_i"]) + "\n"
        engine_text += "def_time: " + str(v["def_time"]) + "\n"


    print("filename: " + str(file))
    if threats == 0:
        print("overall status: " + "Clean")
    else:
        print("overall status: " + " Not Clean")
    print(engine_text)